<a href="https://colab.research.google.com/github/Sciotaku/.github/blob/main/socy34_module1_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Preparatory Code { display-mode: "form" }
#@title  { vertical-output: false, display-mode: "form" }
#import relevant packages
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from google.colab import widgets as g_widgets
from google.colab import output
from matplotlib import pylab
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
!pip install geopandas
import geopandas as gpd
import shapely.geometry
import plotly.express as px
import plotly.graph_objs as go



def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

#load in data from Git repo and TEA website
#data legend: m1 = County,Med. Hhold Income,High school graduation rate,Health Quality2 = County, Physically inactive (2017), Smoking (2018),Health Quality
## m3 = County,Air pollution particulate matter (average daily PM2.5), Severe housing problems (2013-2017),Overcrowding,Health Quality
## m4 = County, Uninsured,Mental health providers 2020 - ratio of pop to providers,Health Quality
cities_url = "https://raw.githubusercontent.com/osmanrkhan/raw_dat_tx_hs_grad_2018/main/uscities.csv"
cities = pd.read_csv(cities_url)
m1_url = 'https://raw.githubusercontent.com/katherinel09/socy34_module1-4/main/socy34_m1_updated_nov.csv'
tea_url = 'https://raw.githubusercontent.com/osmanrkhan/raw_dat_tx_hs_grad_2018/main/raw_dat_tea_hs_grad.csv'

m2_url = 'https://raw.githubusercontent.com/katherinel09/socy34_module1-4/main/socy34_data_texas_m2_UPDATED.csv'
m3_url = 'https://raw.githubusercontent.com/katherinel09/socy34_module1-4/main/socy34_data_texas_m3_UPDATED.csv'
m4_url = 'https://raw.githubusercontent.com/katherinel09/socy34_module1-4/main/socy34_data_texas_m4_UPDATED.csv'






df = pd.read_csv(m1_url)
df = df.rename(columns = {'Poor or fair health': 'Health Quality', 'Median household income overall': 'Med. Hhold Income' })

df_2 = pd.read_csv(m2_url)

df_2 = df_2.rename(columns = {'Poor or fair health': 'Health Quality'})

df_2 = df_2.drop(columns = ['Health Quality'])

df_3 = pd.read_csv(m3_url)
df_3 = df_3.rename(columns = {'Poor or fair health': 'Health Quality'})
df_3 = df_3.drop(columns = ['Health Quality'])

df_4 = pd.read_csv(m4_url)
df_4 = df_4.rename(columns = {'Poor or fair health': 'Health Quality'})
df_4 = df_4.drop(columns = ['Health Quality'])


## load in high school data from Texas Educational Agency website
new_hs_grad_dat = pd.read_csv(tea_url)

#deal with county name/number inconsistencies in data. merge the county health OUTCOME data with the TEA high school grad data Osman scraped

grad = new_hs_grad_dat.drop(columns = ['CNTYNAME'])
mid_dat = pd.merge(df.drop(columns = ['High school graduation rate']), grad,left_on='County', right_on='CNTY_NM',how='left')



# merge in the other three data frames df2, df3, df4
final_dat = pd.merge(pd.merge(pd.merge(mid_dat,
                                       df_2, left_on = "County", right_on = "County", how = 'left'),
                              df_3, left_on = "County", right_on = "County", how = 'left'),
                     df_4, left_on = "County", right_on = "County", how = 'left')

final_dat = final_dat.drop(columns = ['CNTY_NM'])
#now merge that dataframe with Kat's m2, m3, and m4 data, that are legended above



#remove irrelevant fields for analysis
df2 = final_dat.drop(columns=['County'])



final_dat = final_dat.rename(columns={"HS_GRAD": "High school graduation rate"})
feature_names = list(final_dat.columns)


final_dat = final_dat.drop_duplicates(subset = 'County', keep="first")

is_NaN = final_dat.isnull()
row_has_NaN = is_NaN.any(axis=1)



final_dat['High school graduation rate'][final_dat['High school graduation rate'].isnull()] = grad[final_dat['High school graduation rate'].isnull()]['HS_GRAD']


#remove irrelevant fields for analysis
df2 = final_dat.drop(columns=['County'])
feature_names = list(df2.columns)


#remove irrelevant fields for analysis
feature_names = list(df2.columns)

#define dataframes for each covid wave where other waves are excluded
median_household_income	 = df2.drop(columns=['Health Quality', 'High school graduation rate'])

##  OSMAN EDIT HERE: REPLACED HIGH SCHOOL GRAD DATA WITH DATA ACQUIRED FROM
## TEXAS EDUATION AGENCY FOR FOUR YEAR COMPLETION RATE BY COUNTY.
## DATA CAN BE FOUND AT THIS URL: https://tea.texas.gov/reports-and-data/school-performance/accountability-research/completion-graduation-and-dropout/four-year-graduation-and-dropout-data-class-of-2018


poor_or_fair_health	 = df2.drop(columns=[ 'High school graduation rate', 'Med. Hhold Income'])

#read in spatial data for map from Git repo into geopandas dataframe
url2 = 'https://opendata.arcgis.com/datasets/9b2eb7d232584572ad53bad41c76b04d_0.geojson'
county_data = gpd.read_file(url2)
county_data.sort_values(by=['CNTY_NM'], inplace=True)

#define centroids
county_data["center"] = county_data.geometry.centroid

tx_counties = county_data[['CNTY_NM','geometry']].set_index("CNTY_NM")
map_variables = list(county_data.columns)

feature_names_X = list(df2.columns)
feature_names_X.remove('Health Quality')

feature_names_Y = list(df2.columns)
feature_names_Y.remove('High school graduation rate')
feature_names_Y.remove('Med. Hhold Income')

# Scale data to use for bubble layer
data = county_data[[m for m in map_variables]] #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(2,50))
#data_scaled = min_max_scaler.fit_transform(data)
#scaled = pd.DataFrame(data_scaled, columns=feature_names)

# Create data to use for cancer alley borders
county_data_data = county_data[county_data["CNTY_NM"] == 1]
lats = []
lons = []

for feature in county_data_data.geometry.boundary:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)


#define list of variable names for covid wave dropdown in map tab


#define options for turning regression line off and on in regression tab
linear_regr = ['Off','On']

#for var in var_x:
  #df[var+" label"] = df["County"]+": "+df[var].round(2).astype("string")
#  df[var+" label"] = df["County"]+": " # + float(df[var].round(2)).astype("string")

#new_df = county_data["center"].copy()
#new_df['center'] = new_df['center'].buffer(size)
center_circles = county_data["center"]

# EDIT ADDED BY ANDY BEAN ON 4/17/23 to REPAIR MAP -----------------------------
county_data = county_data.reset_index(drop = True)
final_dat = final_dat.reset_index(drop = True)

final_dat_map = county_data.join(final_dat)
# ------------------------------------------------------------------------------


#@title  { vertical-output: false, display-mode: "form" }

#function for creating dropdowns using widget functionality in Colab
def create_dropdown(choices, first, des):
  return widgets.Dropdown(
      options = choices,
      value = choices[first],
      description = des+":",
      disabled=False,
  )

#function for creating scatterplot with linear regression analysis based on selected
#x and y variables
def plot_data_linear(drop1,drop2,drop9):
#error message if user chooses same variable for x and y
  if (drop1 == drop2):
    print("X Variable cannot equal Y Variable")
    return
  f, ax = plt.subplots(figsize= [12,10])
  plt.xlabel(drop1,fontsize=20)
  plt.ylabel(drop2,fontsize=20)
  df3 = df2[[drop1,drop2]]
  df3 = df3.dropna(axis='rows')
  x = df3[[drop1]].values
  y = df3[[drop2]].values
  plt.xlabel(drop1)
  plt.ylabel(drop2)
  regr = LinearRegression()
  regr.fit(x,y)
  y_pred = regr.predict(x)
  y_inter = regr.intercept_.item()
  if np.sign(y_inter) == -1:
    sign = '-'
  else:
    sign = '+'
  slope = regr.coef_.item()
  r2 = r2_score(y, y_pred)
  std_scaler = StandardScaler()
  df_std = pd.DataFrame(std_scaler.fit_transform(df3), columns=df3.columns)
  X1 = df3[[drop1]].values
  Y1 = df3[[drop2]].values
  x1 = X1.flatten()
  y1 = Y1.flatten()
  data = pd.DataFrame({'x': x1, 'y': y1})
  model = ols("y ~ x", data).fit()
  p_value = model.pvalues[1]
  ax.plot(x,y, 'o', c='black')
  if drop9 == 'On':
    ax.plot(x,y_pred, 'r-', label="OLS")
    ax.annotate('R\N{SUPERSCRIPT TWO}= %.2f' % r2,xy=(1.01, .92),xycoords='axes fraction',
                 ha='left',fontsize ='15')
    ax.annotate('y = %.2f' % slope + 'x ' + sign + ' ' '%.2f' % y_inter,xy=(1.01, .96),xycoords='axes fraction',
                 ha='left',fontsize ='15')
    plt.grid(True)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.show()
  else:
    plt.grid(True)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.show()

#function for creating correlation matrix based on chosen covid wave and colorscale
#https://seaborn.pydata.org/examples/many_pairwise_correlations.html

def make_corr_plot(drop3,drop4,drop5):
  sns.set_theme(style="white")
  if drop3 == 'Med. Hhold Income':
    drop3 = median_household_income
  if drop3 == 'High school graduation rate':
    drop3 = hs_grad
  else:
    drop3 = poor_or_fair_health
  std_scaler = StandardScaler()
  df_std = pd.DataFrame(std_scaler.fit_transform(drop3), columns=drop3.columns)
  corr = df_std.corr()
  f, ax = plt.subplots(figsize=(11, 9))
  ax.set_title("Correlation Matrix for Variables", fontsize = 18)
  if drop5 == 'RedBlue':
    cmap = sns.color_palette("vlag", as_cmap=True)
  elif drop5 == 'GreenPurple':
    cmap = sns.diverging_palette(145, 300, s=60, as_cmap=True)
  else:
    cmap = sns.color_palette("icefire", as_cmap=True)
  if drop4 == 'On':
    sns.heatmap(corr, vmin=-1, vmax=1,cmap=cmap,center=0,annot=True, fmt=".2f",
                square=True, linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
  else:
    sns.heatmap(corr, vmin=-1, vmax=1,cmap=cmap,center=0,square=True,
                linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})


#function for creating interactive map that displays covid waves as scaled bubbles and
#chloropleths from chosen variable and colorscale
def draw_map(drop6, drop7, drop8):
  num_scale = 40

  if (drop6 == 'Med. Hhold Income'):
    num_scale = 1000
  elif (drop6 == 'High school graduation rate'):
    num_scale = 1.8
  elif (drop6 == ' Physically inactive (2017)'):
    num_scale = 0.5
  elif (drop6 == ' Smoking (2018)'):
    num_scale = 0.4
  elif (drop6 == 'Air pollution particulate matter (average daily PM2.5)'):
    num_scale = 1/8
  elif (drop6 == ' Severe housing problems (2013-2017)'):
    num_scale = 1/4
  elif (drop6 == 'Overcrowding'):
    num_scale = 1/14
  elif (drop6 == ' Uninsured'):
    num_scale = 1/2.5
  elif (drop6 == 'Mental health providers 2020 - ratio of pop to providers'):
    num_scale = 250


  # Add choropleth layer
  color_dict = {'Viridis':'Viridis',
                'Dark Mint':'DarkMint',
                "Purples": "Purples"}

  # EDIT BY ANDY BEAN 4/17/2023 to REPAIR MAP
  # ----------------------------------------------------------------------------
  # drop 8 is colorscale
  # drop 7 is 3 option variable (shading)
  # drop 6 is 2 option variable (bubbles)

  fig, ax = plt.subplots(figsize=[10,10])

  # create shading
  final_dat_map.plot(column=drop7, cmap = drop8, linewidth = 0.8, ax=ax, edgecolor = '0.8')

  # create bubbles
  bubble_dat = final_dat_map[[drop6]].values

  ax.scatter(final_dat_map.center.x, final_dat_map.center.y, c = "tomato", alpha = 0.5, s = bubble_dat/num_scale)

  for x, y, n in zip(final_dat_map.center.x, final_dat_map.center.y, final_dat_map.CNTY_NM):
    label = f"{n}"
    ax.annotate(label, (x,y), textcoords = 'data', fontsize = 10)

  # create colorbar
  cbot = min(final_dat_map[[drop7]].values)
  ctop = max(final_dat_map[[drop7]].values)
  sm = plt.cm.ScalarMappable(cmap = drop8, norm = plt.Normalize(vmin=cbot, vmax=ctop))
  sm._A = []
  cbar = fig.colorbar(sm)

  # final aesthetics
  ax.set_title('Map View',fontsize = 20)

  plt.show()
  # ----------------------------------------------------------------------------

  #fig.add_trace(go.Choroplethmapbox(geojson=tx_counties.__geo_interface__,
  #                                  locations=county_data.CNTY_NM,
  #                                  z = final_dat[drop7],
  #                                  #colorbar = {'title': drop7},
  #                                  colorbar= {'title': drop7, 'xpad':10,
  #                                             'ypad':100,},
  #                                  colorscale = color_dict.get(drop8),
  #
  #                                  name = drop7
  #                                  ))

  #fig.add_trace(go.Scattermapbox(
  #    lat=center_circles.y,
  #    lon=center_circles.x,
  #    mode='markers',
  #
  #    name = drop6,
  #    text = final_dat['County'].astype(str) + ": " + final_dat[drop6].astype(str),
  #    hoverinfo = 'name+text',
  #    marker=go.scattermapbox.Marker(
  #        size= final_dat[drop6] /num_scale
  #    ),

  #    #hoverlabel = final_dat[str(drop6)],
  #    opacity = 0.45,

  #  ))

  #draw city names
  #fig.add_trace(go.Scattermapbox(
  #  lat=cities['lat'],
  #  lon=cities['lng'],
  #  mode="markers+text",
  #  marker=go.scattermapbox.Marker(
  #        color = "white"
  #    ),
  #  textfont= go.scattermapbox.Textfont(color='black', family='Helvetica', size=300),
  #  text = cities['city'],

  #  hovertemplate=cities['city'] + "<extra></extra>",
  #  showlegend=False
#,
#))


#  fig.update_layout(mapbox_style="carto-positron",
#                   legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),
#                   showlegend=True, mapbox_zoom=4,
#                   mapbox_center = {"lat": 30.9843, "lon": -91.9623-7},
#                   width=1000, height=750, margin={"r":0,"t":0,"l":0,"b":0}
#                   )
  #fig.update(layout_showlegend=False)

#  fig.layout.coloraxis.colorbar.title = str(drop6)
#  fig.show()

def create_heatmap():
  # Build data
  np.random.seed(0) # seed the random number generator in order to make the run repeatable
  dat = 100*np.random.rand(1, 254)
  df_labels = pd.DataFrame(dat)

  # Iterate over numpy array and prepare text labels
  for (row, col), val in np.ndenumerate(dat):
      df_labels.iloc[row, col] = "%.2f%%" % (val)


  plt.figure()
  plt.suptitle("2D percentage values of Health Quality", color='g')
  sns.set(font_scale=0.8)

  ax = sns.heatmap(dat, linewidths=.5, annot=df_labels, fmt="", annot_kws={"size": 12}, vmin=0, vmax=100)
  ax.set(xlabel='County', ylabel='Health outcome (%)')

  plt.show()


<ipython-input-1-c655a2916bf3>:109: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_dat['High school graduation rate'][final_dat['High school graduation rate'].isnull()] = grad[final_dat['High school graduation rate'].isnull()]['HS_GRAD']
<ipython-input-1-c655a2916bf3>:136: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  county_data["center"] = county_data.geometry.centroid


# **Welcome to SOCY 34's Interactive Data Component! **

Here's how to use this notebook:

Use the arrow button in the top left corner of the code blocks to run the code.


Make sure to run the blocks in order.

When changing views or running the initial large code block, be patient. It could take a few seconds to load.

Once you've opened the visuzaliations, make sure to click on the legend, zoom in and out, and mouse over both the circular data and the county data!

Clicking on the legend will hide things you want to ignore for a moment.

Try playing with the drop-downs, and swap between the chloropleth and the scatter plots.

Good luck!


**Run the small code block right after this line to get a quick run-through of all the data available to you over the modules in this course! If there's a term you don't understand, look it up!** Any time you're wondering if a specific county is an outlier, come back and check out this table.

Finally, remember data isn't perfect! There are many things that county level data can't catch!

In [2]:
display(final_dat.describe())

,Med. Hhold Income,Health Quality,High school graduation rate,Physically inactive (2017),Smoking (2018),Air pollution particulate matter (average daily PM2.5),Severe housing problems (2013-2017),Overcrowding,Uninsured,Mental health providers 2020 - ratio of pop to providers
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000
mean,54839.370079,76.645669,93.861811,25.429134,18.984252,7.365748,14.055118,4.165354,20.944882,2793.070866
std,13497.042027,4.953583,5.505032,4.963991,2.342203,1.187937,4.008503,2.576156,4.000607,3344.935766
min,23100.000000,59.000000,59.900000,15.000000,12.000000,5.200000,0.000000,0.000000,11.000000,0.000000
25%,45700.000000,75.000000,92.150000,22.000000,18.000000,6.300000,12.000000,3.000000,18.000000,732.500000
50%,52600.000000,77.000000,95.000000,25.000000,19.000000,7.300000,14.000000,4.000000,21.000000,1770.000000
75%,61350.000000,80.000000,97.275000,28.750000,21.000000,8.475000,16.000000,5.000000,23.000000,3545.000000
max,105800.000000,86.000000,100.000000,41.000000,24.000000,10.400000,29.000000,14.000000,32.000000,20080.000000


from matplotlib import pyplot as plt
_df_0['Med. Hhold Income'].plot(kind='hist', bins=20, title='Med. Hhold Income')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
_df_1['Health Quality'].plot(kind='hist', bins=20, title='Health Quality')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
_df_2['High school graduation rate'].plot(kind='hist', bins=20, title='High school graduation rate')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
_df_3[' Physically inactive (2017)'].plot(kind='hist', bins=20, title=' Physically inactive (2017)')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.tight_layout()

...

from matplotlib import pyplot as plt
_df_12[' Physically inactive (2017)'].plot(kind='line', figsize=(8, 4), title=' Physically inactive (2017)')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='Med. Hhold Income', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Health Quality', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='High school graduation rate', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x=' Physically inactive (2017)', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [3]:
#@title MODULE 1: Socioeconomic Factors
#module 1 loadout

var_x = ['Med. Hhold Income', 'High school graduation rate']
var_y = ['Health Quality']


#define list of variable names for colorscales
color_scales = ["viridis", "inferno", "GnBu", "Reds", "RdBu", "BuRd"]

#define list of variable names for covid wave dropdown in correlation matrix tab


#function for creating the different tabs and creating content in each tab


#create dropdowns for all options in each tab
widget1 = create_dropdown(var_x, 0, "X Variable")
widget2 = create_dropdown(np.append(var_y, var_x), 0, "Y Variable")



widget4 = create_dropdown(['On','Off'], 1, "R Values")
widget5 = create_dropdown(['RedBlue','GreenPurple','BlueOrange'], 0, "Colorbar")
widget6 = create_dropdown(var_x, 0, "")
widget7 = create_dropdown(np.append(var_y, var_x), 0, "County Data")
widget8 = create_dropdown(color_scales, 2, "Colorbar")
widget9 = create_dropdown(linear_regr,0,"Regression")



def create_tab(location):
  tb = g_widgets.TabBar(['Linear Regression','Chloropleth'], location=location)
  with tb.output_to('Linear Regression'):
    pylab.figure()
    widgets.interact(plot_data_linear, drop1=widget1,drop2=widget2,drop9=widget9)
  with tb.output_to('Chloropleth'):
    pylab.figure()
    widgets.interact(draw_map, drop6=widget6,drop7=widget7,drop8=widget8)


create_tab('top')


/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py:711: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description=':', options=('Med. Hhold Income', 'High school graduation rate'), …

<IPython.core.display.Javascript object>

In [4]:
#@title MODULE 2: Health Behaviors
#module 2 loadout


var_x = [' Physically inactive (2017)', ' Smoking (2018)']
var_y = ['Health Quality']



#define list of variable names for colorscales
color_scales = ["viridis", "inferno", "GnBu", "Reds", "RdBu", "BuRd"]

#define list of variable names for covid wave dropdown in correlation matrix tab


#function for creating the different tabs and creating content in each tab


#create dropdowns for all options in each tab
widget1 = create_dropdown(var_x, 0, "X Variable")
widget2 = create_dropdown(np.append(var_y, var_x), 0, "Y Variable")

widget4 = create_dropdown(['On','Off'], 1, "R Values")
widget5 = create_dropdown(['RedBlue','GreenPurple','BlueOrange'], 0, "Colorbar")
widget6 = create_dropdown(var_x, 0, "X")
widget7 = create_dropdown(np.append(var_y, var_x), 0, "Y")
widget8 = create_dropdown(color_scales, 2, "Colorbar")
widget9 = create_dropdown(linear_regr,0,"Regression")



def create_tab(location):
  tb = g_widgets.TabBar(['Linear Regression','Chloropleth'], location=location)
  with tb.output_to('Linear Regression'):
    pylab.figure()
    widgets.interact(plot_data_linear, drop1=widget1,drop2=widget2,drop9=widget9)
  with tb.output_to('Chloropleth'):
    pylab.figure()
    widgets.interact(draw_map, drop6=widget6,drop7=widget7,drop8=widget8)


create_tab('top')

/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py:711: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description='X:', options=(' Physically inactive (2017)', ' Smoking (2018)'), v…

<IPython.core.display.Javascript object>

In [5]:
#@title MODULE 3: Physical Environment Factors
#module 3 loadout

var_x = ['Air pollution particulate matter (average daily PM2.5)',
       ' Severe housing problems (2013-2017)', 'Overcrowding']
var_y = ['Health Quality']



#define list of variable names for colorscales
color_scales = ["viridis", "inferno", "GnBu", "Reds", "RdBu", "BuRd"]

#define list of variable names for covid wave dropdown in correlation matrix tab

#function for creating the different tabs and creating content in each tab


#create dropdowns for all options in each tab
widget1 = create_dropdown(var_x, 0, "X Variable")
widget2 = create_dropdown(np.append(var_y, var_x), 0, "Y Variable")

widget4 = create_dropdown(['On','Off'], 1, "R Values")
widget5 = create_dropdown(['RedBlue','GreenPurple','BlueOrange'], 0, "Colorbar")
widget6 = create_dropdown(var_x, 0, "X")
widget7 = create_dropdown(np.append(var_y, var_x), 0, "Y")
widget8 = create_dropdown(color_scales, 2, "Colorbar")
widget9 = create_dropdown(linear_regr,0,"Regression")



def create_tab(location):
  tb = g_widgets.TabBar(['Linear Regression','Chloropleth'], location=location)
  with tb.output_to('Linear Regression'):
    pylab.figure()
    widgets.interact(plot_data_linear, drop1=widget1,drop2=widget2,drop9=widget9)
  with tb.output_to('Chloropleth'):
    pylab.figure()
    widgets.interact(draw_map, drop6=widget6,drop7=widget7,drop8=widget8)


create_tab('top')

/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py:711: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description='X:', options=('Air pollution particulate matter (average daily PM2…

<IPython.core.display.Javascript object>

In [6]:
#@title MODULE 4: Clinical Access
#module 4 loadout




var_x = [' Uninsured',
       'Mental health providers 2020 - ratio of pop to providers']
var_y = ['Health Quality']



#define list of variable names for colorscales
color_scales = ["viridis", "inferno", "GnBu", "Reds", "RdBu", "BuRd"]

#define list of variable names for covid wave dropdown in correlation matrix tab

#function for creating the different tabs and creating content in each tab


#create dropdowns for all options in each tab
widget1 = create_dropdown(var_x, 0, "X Variable")
widget2 = create_dropdown(np.append(var_y, var_x), 0, "Y Variable")


widget4 = create_dropdown(['On','Off'], 1, "R Values")
widget5 = create_dropdown(['RedBlue','GreenPurple','BlueOrange'], 0, "Colorbar")
widget6 = create_dropdown(var_x, 0, "X")
widget7 = create_dropdown(np.append(var_y, var_x), 0, "Y")
widget8 = create_dropdown(color_scales, 2, "Colorbar")
widget9 = create_dropdown(linear_regr,0,"Regression")



def create_tab(location):
  tb = g_widgets.TabBar(['Linear Regression','Chloropleth'], location=location)
  with tb.output_to('Linear Regression'):
    pylab.figure()
    widgets.interact(plot_data_linear, drop1=widget1,drop2=widget2,drop9=widget9)
  with tb.output_to('Chloropleth'):
    pylab.figure()
    widgets.interact(draw_map, drop6=widget6,drop7=widget7,drop8=widget8)


create_tab('top')

/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py:711: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description='X:', options=(' Uninsured', 'Mental health providers 2020 - ratio …

<IPython.core.display.Javascript object>

##<u>Credits</u>
**SOCY 34 Team:** Katherine Lasonde ('23), Kyra McLaughlin ('23), Emily Walton (Professor of Sociology) Scott Pauls (PI)

**DIFUSE Project Managers:** Taylor Hickey ('23)

**DIFUSE PI's:** Prof. Petra Bonfert-Taylor (Thayer School), Prof. Laura Ray (Thayer School), Prof. Scott Pauls (Mathematics), Prof. Lori Loeb (Computer Science)

**Development Team:** James Busch compiled and authored code for the web application, correlation matrix, and linear regression components. William Chen compiled and authored the code used for the interactive map. Osman Khan shipped the product and debugged the module.

*The DIFUSE project is supported by the National Science Foundation under grant no. DUE- 1917002*

*Thank you to the ENVS 3 DIFUSE team for the basis of the modules!!*



```
# This is formatted as code
```

